In [1]:
import torch
print(torch.__version__)

1.12.1+cu102


In [2]:
# f1 = open('ctrain.csv', 'w+')
# for c in range(20, 100):
#     f =  1.8*c + 32
#     f1.writelines(str(c) + "," + str(f) + "\n")
# f1.close()



In [3]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')
train_df.head()

,c,f
0,20,68.0
1,21,69.8
2,22,71.6
3,23,73.4
4,24,75.2


In [4]:
from torch.utils.data import Dataset, DataLoader

class createDataset(Dataset):
    def __init__(self, df):
        self.features = df[['c']]
        self.output = df[["f"]]
    
    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return {"input":float(self.features.iloc[index]), "output":float(self.output.iloc[index])}

In [5]:
train_dlo = createDataset(train_df)
val_dlo = createDataset(val_df)

batch_size = 16
train_dataloader = DataLoader(train_dlo, shuffle=False, batch_size = batch_size)
val_dataloader = DataLoader(val_dlo, shuffle=False, batch_size = batch_size)

def visualize(dl):
    global batch_size
    for i,sample in enumerate(dl):
        for j in range(batch_size):
            print(sample["input"][j], sample["output"][j])
        break

visualize(train_dataloader)


tensor(20., dtype=torch.float64) tensor(68., dtype=torch.float64)
tensor(21., dtype=torch.float64) tensor(69.8000, dtype=torch.float64)
tensor(22., dtype=torch.float64) tensor(71.6000, dtype=torch.float64)
tensor(23., dtype=torch.float64) tensor(73.4000, dtype=torch.float64)
tensor(24., dtype=torch.float64) tensor(75.2000, dtype=torch.float64)
tensor(25., dtype=torch.float64) tensor(77., dtype=torch.float64)
tensor(26., dtype=torch.float64) tensor(78.8000, dtype=torch.float64)
tensor(27., dtype=torch.float64) tensor(80.6000, dtype=torch.float64)
tensor(28., dtype=torch.float64) tensor(82.4000, dtype=torch.float64)
tensor(29., dtype=torch.float64) tensor(84.2000, dtype=torch.float64)
tensor(30., dtype=torch.float64) tensor(86., dtype=torch.float64)
tensor(31., dtype=torch.float64) tensor(87.8000, dtype=torch.float64)
tensor(32., dtype=torch.float64) tensor(89.6000, dtype=torch.float64)
tensor(33., dtype=torch.float64) tensor(91.4000, dtype=torch.float64)
tensor(34., dtype=torch.float64)

In [6]:
visualize(val_dataloader)

tensor(101., dtype=torch.float64) tensor(213.8000, dtype=torch.float64)
tensor(102., dtype=torch.float64) tensor(215.6000, dtype=torch.float64)
tensor(103., dtype=torch.float64) tensor(217.4000, dtype=torch.float64)
tensor(104., dtype=torch.float64) tensor(219.2000, dtype=torch.float64)
tensor(105., dtype=torch.float64) tensor(221., dtype=torch.float64)
tensor(106., dtype=torch.float64) tensor(222.8000, dtype=torch.float64)
tensor(107., dtype=torch.float64) tensor(224.6000, dtype=torch.float64)
tensor(108., dtype=torch.float64) tensor(226.4000, dtype=torch.float64)
tensor(109., dtype=torch.float64) tensor(228.2000, dtype=torch.float64)
tensor(110., dtype=torch.float64) tensor(230., dtype=torch.float64)
tensor(111., dtype=torch.float64) tensor(231.8000, dtype=torch.float64)
tensor(112., dtype=torch.float64) tensor(233.6000, dtype=torch.float64)
tensor(113., dtype=torch.float64) tensor(235.4000, dtype=torch.float64)
tensor(114., dtype=torch.float64) tensor(237.2000, dtype=torch.float64)


In [7]:
import torch.nn as nn
import torch.nn.functional as F 

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(in_features=1, out_features=1)
    def forward(self, x):
        return self.layer1(x)


my_model = Model()
from torchsummary import summary
print(summary(my_model, input_size=(batch_size,1)))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 16, 1]               2
Total params: 2
Trainable params: 2
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------
None


In [8]:
#!pip install torchsummary

In [27]:
import torch.optim as optim

optimizer = optim.Adam(my_model.parameters(), lr=0.03)
criterion = nn.L1Loss()
epochs = 50

In [10]:
def train_model(model, batch_size, optim1, criterion, epochs, dataloader):
    model.train()
    for i in range(epochs):
        epoch_loss = 0.0
        for j, sample in enumerate(dataloader):
            #print(len(sample))
            inputs = sample['input']
            outputs = sample['output']
            #print(inputs, outputs)
        #for k in range(batch_size):
            optimizer.zero_grad()
            xinput = inputs.reshape((16,1)) #[k]
            #print(xinput, xinput.shape)
            actual_output = outputs#[k]
            model_output = model(xinput.float())
            loss = criterion(model_output, actual_output)
            print(loss)
            #print(model_output)
            loss.backward()
            optim1.step()
        
    return model


In [28]:
train_model(model=my_model, batch_size=batch_size, optim1=optimizer, criterion=criterion, epochs=300,
dataloader=train_dataloader)

tensor(9.5698, grad_fn=<L1LossBackward0>)
tensor(9.3508, grad_fn=<L1LossBackward0>)
tensor(9.6288, grad_fn=<L1LossBackward0>)
tensor(10.0162, grad_fn=<L1LossBackward0>)
tensor(10.1107, grad_fn=<L1LossBackward0>)
tensor(9.4976, grad_fn=<L1LossBackward0>)
tensor(9.4999, grad_fn=<L1LossBackward0>)
tensor(9.5186, grad_fn=<L1LossBackward0>)
tensor(9.5291, grad_fn=<L1LossBackward0>)
tensor(9.5209, grad_fn=<L1LossBackward0>)
tensor(9.7690, grad_fn=<L1LossBackward0>)
tensor(9.6397, grad_fn=<L1LossBackward0>)
tensor(9.5200, grad_fn=<L1LossBackward0>)
tensor(9.4415, grad_fn=<L1LossBackward0>)
tensor(9.5214, grad_fn=<L1LossBackward0>)
tensor(9.5441, grad_fn=<L1LossBackward0>)
tensor(9.4222, grad_fn=<L1LossBackward0>)
tensor(9.4139, grad_fn=<L1LossBackward0>)
tensor(9.5956, grad_fn=<L1LossBackward0>)
tensor(9.9085, grad_fn=<L1LossBackward0>)
tensor(9.4576, grad_fn=<L1LossBackward0>)
tensor(9.3952, grad_fn=<L1LossBackward0>)
tensor(9.4092, grad_fn=<L1LossBackward0>)
tensor(9.4836, grad_fn=<L1LossBa

Model(
  (layer1): Linear(in_features=1, out_features=1, bias=True)
)

In [29]:
for q in range(-3, 17):
    print(my_model(torch.Tensor([q], device=torch.device('cpu'))), 1.8*q + 32)


tensor([30.5406], grad_fn=<AddBackward0>) 26.6
tensor([32.2795], grad_fn=<AddBackward0>) 28.4
tensor([34.0184], grad_fn=<AddBackward0>) 30.2
tensor([35.7574], grad_fn=<AddBackward0>) 32.0
tensor([37.4963], grad_fn=<AddBackward0>) 33.8
tensor([39.2352], grad_fn=<AddBackward0>) 35.6
tensor([40.9741], grad_fn=<AddBackward0>) 37.4
tensor([42.7130], grad_fn=<AddBackward0>) 39.2
tensor([44.4520], grad_fn=<AddBackward0>) 41.0
tensor([46.1909], grad_fn=<AddBackward0>) 42.8
tensor([47.9298], grad_fn=<AddBackward0>) 44.6
tensor([49.6687], grad_fn=<AddBackward0>) 46.4
tensor([51.4076], grad_fn=<AddBackward0>) 48.2
tensor([53.1466], grad_fn=<AddBackward0>) 50.0
tensor([54.8855], grad_fn=<AddBackward0>) 51.8
tensor([56.6244], grad_fn=<AddBackward0>) 53.6
tensor([58.3633], grad_fn=<AddBackward0>) 55.400000000000006
tensor([60.1022], grad_fn=<AddBackward0>) 57.2
tensor([61.8412], grad_fn=<AddBackward0>) 59.0
tensor([63.5801], grad_fn=<AddBackward0>) 60.8
